## KEPLER
More details about KEPLER you can find them in this link:
https://github.com/THU-KEG/KEPLER

In [3]:
import os
os.environ["WANDB_MODE"] = "dryrun"

In [4]:
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import pandas as pd

In [5]:
# specify the directory where model files are saved
model_dir = "./models/KEPLER/KEPLER_INPUT/"
data_set= "../data/microtext_references.pickle" # "../data/kialo_references.pickle"

# Load the model
tokenizer = RobertaTokenizer.from_pretrained(model_dir)
config = RobertaConfig.from_pretrained(model_dir)
kepler_model = RobertaForSequenceClassification.from_pretrained(model_dir,
                                                            config=config)

Some weights of the model checkpoint at ./models/KEPLER/KEPLER_INPUT/ were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./models/KEPLER/KEPLER_INPUT/ and are newly initialized: ['cl

In [6]:
# Program
torch.manual_seed(0)

class RelationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __getitem__(self, index):
        premise = self.data["premise"].iloc[index]
        claim = self.data["claim"].iloc[index]

        encoding = self.tokenizer.encode_plus(
            premise,
            claim,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )


        if 'label' in self.data.columns:
            
            label = torch.tensor(0 if self.data["label"].iloc[index] == "Attack" else 1, dtype=torch.int64)
            
            return {
            'input_ids': encoding['input_ids'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
         }
            
        else:
            return {
            'input_ids': encoding['input_ids'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
                    

    def __len__(self):
        return len(self.data)

In [7]:
def create_dataset(mode: str, tokenizer, shuffle=False):
    
    df = pd.read_pickle(data_set)
    split = df[df['mode'] == mode]
    split = split[split['label'] != 'Support: Example']
    #split.reset_index(drop=True)
    
    return RelationDataset(split, tokenizer)

In [8]:
train_dataset = create_dataset("train", tokenizer, True)
validate_dataset = create_dataset("validate", tokenizer, False)

In [9]:
# Load datasets
df = pd.read_pickle(data_set)

train_df = df[df['mode'] == 'train']
labels = train_df[train_df['label'] != 'Support: Example']['label']
print('Train dataset class distribution:\n', labels.value_counts())

Train dataset class distribution:
 label
Support    211
Attack      78
Name: count, dtype: int64


In [10]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(labels),
                                        y = labels                                                    
                                    )
dic_class_weights = dict(zip(np.unique(labels), class_weights))
print("Class weights: ", dic_class_weights)

# Convert the list to a tensor
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights.to(device)

Class weights:  {'Attack': 1.8525641025641026, 'Support': 0.6848341232227488}


In [12]:
from torch.nn import CrossEntropyLoss

def compute_metrics(pred):
    if isinstance(pred.predictions, tuple):
        preds = pred.predictions[0].argmax(-1)
    else:
        preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    
class CustomTrainer(Trainer):
    def __init__(self, model, args, train_dataset, loss_fct, **kwargs):
        super().__init__(model, args, train_dataset=train_dataset, **kwargs)
        self.loss_fct = loss_fct

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

# Define your loss function
loss_fct = CrossEntropyLoss(weight=class_weights)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for the learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=3e-5,
    #logging_dir='./logs',
    logging_steps=10,
)

# Initialize your trainer with your custom loss function
trainer = CustomTrainer(
    model=kepler_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    compute_metrics=compute_metrics,
    loss_fct=loss_fct,
)

# Train the model
trainer.train()

/home/afigueroa/tfIntegration/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.663500
20,0.713200
30,0.675600
40,0.668400
50,0.716500
60,0.662400
70,0.697200
80,0.712100
90,0.690500
100,0.668700


TrainOutput(global_step=365, training_loss=0.6948241109717382, metrics={'train_runtime': 31.6919, 'train_samples_per_second': 45.595, 'train_steps_per_second': 11.517, 'total_flos': 95048868748800.0, 'train_loss': 0.6948241109717382, 'epoch': 5.0})

In [16]:
def compute_metrics(pred):
    if isinstance(pred.predictions, tuple):
        preds = pred.predictions[0].argmax(-1)
    else:
        preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

    
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=0.01,
    logging_dir='./logs',            
    logging_steps=10,
)

# Create the Trainer and train
trainer = Trainer(
    model=kepler_model,                   # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=validate_dataset,             # evaluation dataset
    compute_metrics=compute_metrics      
)

trainer.train()

/home/afigueroa/tfIntegration/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.702800
20,0.594100
30,0.624500
40,0.606700
50,0.688700
60,0.797600
70,0.688000
80,0.536800
90,0.607200


TrainOutput(global_step=95, training_loss=0.6511028616051925, metrics={'train_runtime': 14.0499, 'train_samples_per_second': 106.051, 'train_steps_per_second': 6.762, 'total_flos': 98008868121600.0, 'train_loss': 0.6511028616051925, 'epoch': 5.0})

In [14]:
from sklearn.metrics import classification_report

mapping = {'Attack': 0, 'Support': 1}

# Load the test dataset
df = pd.read_pickle("../data/microtext_references.pickle")
split = df[df['mode'] == 'test']
split['label'] = split['label'].map(mapping)

test_dataset = RelationDataset(split, tokenizer)

# Make predictions
raw_pred, _, _ = trainer.predict(test_dataset)
#preds = np.argmax(raw_pred[0], axis=1)
preds = raw_pred.argmax(axis=1)

# Print classification report
report = classification_report(split['label'].values, preds)
print(report)


/tmp/ipykernel_9500/2293473177.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split['label'] = split['label'].map(mapping)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.69      1.00      0.82        34

    accuracy                           0.69        49
   macro avg       0.35      0.50      0.41        49
weighted avg       0.48      0.69      0.57        49



/home/afigueroa/tfIntegration/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/afigueroa/tfIntegration/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/afigueroa/tfIntegration/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [20]:
print(type(raw_pred))
print([type(i) for i in raw_pred])


<class 'tuple'>
[<class 'numpy.ndarray'>, <class 'tuple'>]


In [15]:
# Save the model
trainer.save_model("./models/KEPLER/KEPLER_MICRO/")

In [ ]:
# specify the directory where model files are saved
model_dir = "./models/KEPLER/KEPLER_INPUT/"
data_set= "../data/microtext_references.pickle" # "../data/kialo_references.pickle"

# Load the model
tokenizer = RobertaTokenizer.from_pretrained(model_dir)
config = RobertaConfig.from_pretrained(model_dir)
kepler_model = RobertaForSequenceClassification.from_pretrained(model_dir,
                                                            config=config)

# Program
torch.manual_seed(0)

class RelationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __getitem__(self, index):
        premise = self.data["premise"].iloc[index]
        claim = self.data["claim"].iloc[index]

        encoding = self.tokenizer.encode_plus(
            premise,
            claim,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )


        if 'label' in self.data.columns:
            
            label = torch.tensor(0 if self.data["label"].iloc[index] == "Attack" else 1, dtype=torch.int64)
            
            return {
            'input_ids': encoding['input_ids'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
         }
            
        else:
            return {
            'input_ids': encoding['input_ids'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
                    

    def __len__(self):
        return len(self.data)

def create_dataset(mode: str, tokenizer, shuffle=False):
    
    df = pd.read_pickle(data_set)
    split = df[df['mode'] == mode]
    split = split[split['label'] != 'Support: Example']
    #split.reset_index(drop=True)
    
    return RelationDataset(split, tokenizer)

train_dataset = create_dataset("train", tokenizer, True)
validate_dataset = create_dataset("validate", tokenizer, False)

# Load datasets
df = pd.read_pickle(data_set)

train_df = df[df['mode'] == 'train']
labels = train_df[train_df['label'] != 'Support: Example']['label']
print('Train dataset class distribution:\n', labels.value_counts())

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(labels),
                                        y = labels                                                    
                                    )
dic_class_weights = dict(zip(np.unique(labels), class_weights))
print("Class weights: ", dic_class_weights)

# Convert the list to a tensor
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights.to(device)

from torch.nn import CrossEntropyLoss

def compute_metrics(pred):
    if isinstance(pred.predictions, tuple):
        preds = pred.predictions[0].argmax(-1)
    else:
        preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    
class CustomTrainer(Trainer):
    def __init__(self, model, args, train_dataset, loss_fct, **kwargs):
        super().__init__(model, args, train_dataset=train_dataset, **kwargs)
        self.loss_fct = loss_fct

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

# Define your loss function
loss_fct = CrossEntropyLoss(weight=class_weights)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for the learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=3e-5,
    #logging_dir='./logs',
    logging_steps=10,
)

# Initialize your trainer with your custom loss function
trainer = CustomTrainer(
    model=kepler_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    compute_metrics=compute_metrics,
    loss_fct=loss_fct,
)

# Train the model
trainer.train()